<a href="https://colab.research.google.com/github/MiguelAngeloTr/BIGDATA/blob/main/C2/P2/Parcial_2_BIGDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Universidad Autónoma de Occidente

##Facultad de Ingenierías y Ciencias Básicas

##Curso: Big Data

##Profesor: Juan Sebastian Mosquera

##Correo: smosquera@uao.edu.co

---

### **NOMBRE COMPLETO:**

###**CODIGO:**

### **PROGRAMA:**

---

### **Condiciones**

*   Pueden usar los archivos Google Colab trabajados en clase.
*   Utilizar PySpark para el procesamiento de datos y aplicación de la tarea de ML, para gráficos pueden migrar a pandas unicamente las variables seleccionadas.
*   No pueden usar IA para buscar las respuetas del examen. Quien sea visto en flagrancia, el examen será **anulado**.
*   Pueden usar el material de referencia de UAO virtual
*   Subir el archivo con extensión .ipynb a la plataforma antes de las 11:59 PM.

# **Examen 2**

Se presentan dos conjuntos de datos con multiples características. Se requiere llevar a cabo una tarea de regresión y una de clasificación con cada uno de ellos, es decir, deberá elegir que tarea resolver para el primer dataset y realizar la otra tarea para el segundo.

## **Dataset 1**

El conjunto de datos es el resultado de un análisis de la estabilidad local de un sistema en estrella de 4 nodos (el productor de electricidad está en el centro) aplicando el concepto de control descentralizado de una red inteligente.

Este conjunto comprende 10.000 registros, que permiten determinar la estabilidad del sistema

- tau[x]: reaction time of participant (real from the range [0.5,10]s). Tau1 - the value for electricity producer.
- p[x]: nominal power consumed(negative)/produced(positive)(real). For consumers from the range [-0.5,-2]s^-2; p1 = abs(p2 + p3 + p4)
- g[x]: coefficient (gamma) proportional to price elasticity (real from the range [0.05,1]s^-1). g1 - the value for electricity producer.
- **stab:** the maximal real part of the characteristic equation root (if positive - the system is linearly unstable)(real)
- **stabf:** the stability label of the system (categorical: stable/unstable)

## **Dataset 2**

El conjunto de datos continee datos provenientes de un acelerómetro de las vibraciones de un ventilador con pesos en sus aspas.

Para generar vibraciones se utilizó un ventilador con pesas en las aspas. A este ventilador se le acopló un acelerómetro para recoger los datos de las vibraciones. Con estos datos, se hicieron predicciones del tiempo de fallo del motor, utilizando una red neuronal artificial. Para generar tres escenarios de vibración distintos, los pesos se distribuyeron de tres maneras diferentes: 1) «rojo» - configuración normal: dos piezas de peso colocadas en palas vecinas; 2) “azul” - configuración perpendicular: dos piezas de peso colocadas en palas que forman un ángulo de 90°; 3) «verde» - configuración opuesta: dos piezas de peso colocadas en palas opuestas.

Se establecieron 17 velocidades de rotación, entre el 20% y el 100% de la velocidad máxima del ventilador a intervalos del 5%; para las tres configuraciones de distribución de pesos en las palas del ventilador.

Las mediciones de vibración se recogieron a una frecuencia de 20 ms durante 1 min para cada porcentaje, generando 3000 registros por velocidad. Así, en total, se recogieron 153.000 registros de vibración del modelo de simulación.

Las variables registradas son las siguientes:

- **wconfid:** Weight Configuration ID (1 - 'red' - normal configuration; 2 - 'blue' - perpendicular configuration; 3 - 'green' - opposite configuration)
- **pctid:** Cooler Fan RPM Speed Percentage ID (20 means 20%, and so on).
- x: Accelerometer x value.
- y: Accelerometer y value.
- z: Accelerometer z value.

### Realice el ajuste a los modelo de ML seleccionados teniendo en cuenta todo el flujo de trabajo visto en clase:

* Preprocesado de datos (Vacios, columnas innecesarias, entre otros)
* Analisis Exploratorio de Datos (Bien realizada y con escritos apoyados en datos de que sucede en este dataset)
* Seleccion de Características (al menos deben ser 2, maximo las que consideren necesarias. Todo debidamente soportado)
* Encoding de variables categoricas si aplica.
* Entrenamiento del Modelo
* Validación del Modelo

Finalmente se esperan conclusiones sobre el modelo, donde determinen si los resultados son confiables o no.

In [3]:


!pip install -q pyspark
!pip install -q findspark


In [4]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.window as W
import pyspark.sql.types as T
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
spark = SparkSession.builder.appName("AED PySpark").getOrCreate()
spark


In [11]:
#cargar dataset1

import requests
path = "https://raw.githubusercontent.com/MiguelAngeloTr/BIGDATA/refs/heads/main/C2/P2/dataset1.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'dataset1.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()



#cargar dataset2

import requests
path = "https://raw.githubusercontent.com/MiguelAngeloTr/BIGDATA/refs/heads/main/C2/P2/dataset2.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'dataset2.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

In [12]:
df1 = spark.read.csv('/content/dataset1.csv', header=True, inferSchema=True)
print('Shape of the dataset: ', (df1.count(), len(df1.columns)))
df1.show(n=10)
df2 = spark.read.csv('/content/dataset2.csv', header=True, inferSchema=True)
print('Shape of the dataset: ', (df2.count(), len(df2.columns)))
df2.show(n=10)

Shape of the dataset:  (10000, 14)
+-----------------+----------------+----------------+----------------+----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+--------------------+--------+
|             tau1|            tau2|            tau3|            tau4|              p1|                p2|                p3|                p4|               g1|                g2|               g3|               g4|                stab|   stabf|
+-----------------+----------------+----------------+----------------+----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+--------------------+--------+
| 2.95906002455997|3.07988520422811|8.38102539191882|9.78075443222607|3.76308477206316|-0.782603630987543| -1.25739482958732|  -1.7230863114883|0.650456460887227| 0.859578105752345|0.887444920638513|0.958033987602737|  0.